# Deep Learning: Charity Funding Predictor

## Contributor: Sam Espe

### Part 1: Pre-Processing Data

In [39]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

In [40]:
#  Import and read the charity_data.csv. The CSV lives in an S3 bucket I made.

application_df = pd.read_csv('https://charity-funding-data-set.s3.us-east-2.amazonaws.com/charity_data.csv')
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [41]:
# Split off IS_SUCCESSFUL (target) from the feature variables
y = application_df["IS_SUCCESSFUL"]
X = application_df.drop(columns= "IS_SUCCESSFUL")

In [42]:
# Remove unnecessary columns (EIN & NAME)
X = X.drop(columns= ["EIN", "NAME"])

In [43]:
# Determine number of unique values for each column
X.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
dtype: int64

For non-numerical columns with more than 10 unique values (Application Type and Classification), bin low-frequency values into an "Other" category.

In [44]:
# Find distribution of unique values of Application Type
X["APPLICATION_TYPE"].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [45]:
# Save the value counts series

app_type = X["APPLICATION_TYPE"].value_counts()

In [46]:
# Turn the series into a dictionary

app_dict = app_type.to_dict()

In [47]:
# If the application type has fewer than 100 instances, add it to the list of low-frequency application types.

small_apps = []

for key, value in app_dict.items():
  if int(value) < 100:
    small_apps.append(key)

In [48]:
# If an entry in the Application Type column of the dataframe contains an application type that is in the low-frequency list, replace the value with "Other".

for line in range(len(X)):
    if X.iloc[line, 0] in small_apps:
        X.iloc[line, 0] = "Other"

In [49]:
# Find the distribution of unique values of Classification

X["CLASSIFICATION"].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [50]:
# Save the value counts of the unique classifications

classes = X["CLASSIFICATION"].value_counts()

In [51]:
# Turn the series into a dictionary

classes_dict = classes.to_dict()

In [52]:
# If the class has fewer than 200 instances, add the class to the list of low-frequency classes.

small_classes = []

for key, value in classes_dict.items():
    if int(value) < 200:
        small_classes.append(key)

In [53]:
# If an entry in the Classification column of the dataframe contains a classification that is in the low-frequency list, replace the value with "Other".

for line in range(len(X)):
    if X.iloc[line, 2] in small_classes:
        X.iloc[line, 2] = "Other"

In [54]:
# One-hot encode categorical variables (Application Type, Classification, Use Case, Organizaton Type, Income Amt, Special Considerations)

X_encoded_df = pd.get_dummies(X)

In [55]:
# The Special Considerations column was encoded twice: once where Y was encoded as 1, and once where N was encoded as 1. I am removing the column where N was encoded as 1 to
# avoid having duplicate data.

X_dummies = X_encoded_df.drop(columns = "SPECIAL_CONSIDERATIONS_N")

In [56]:
# Split data set into training and testing sets

X_train, X_test, y_train, y_test = train_test_split(X_encoded_df, y, random_state = 7)

In [57]:
# Scale the data

scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler()

In [58]:
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

### Step 2: Compile, Train, and Evaluate the Model

In [59]:
# Create Keras sequential model

nn_model = tf.keras.models.Sequential()

In [60]:
# Create input layer

nn_model.add(tf.keras.layers.Dense(units = 150, activation = "relu", input_dim = 46))

In [61]:
# Create first hidden layer

nn_model.add(tf.keras.layers.Dense(units = 150, activation = "relu"))

In [62]:
# Create output layer

nn_model.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

In [63]:
# Check structure of the model

nn_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 150)               7050      
                                                                 
 dense_4 (Dense)             (None, 150)               22650     
                                                                 
 dense_5 (Dense)             (None, 1)                 151       
                                                                 
Total params: 29,851
Trainable params: 29,851
Non-trainable params: 0
_________________________________________________________________


In [64]:
# Compile the model

nn_model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

In [65]:
# Fit the model to the training set

fit_model = nn_model.fit(X_train_scaled, y_train, epochs = 100)

Epoch 1/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5701 - accuracy: 0.7223
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5537 - accuracy: 0.7303
Epoch 3/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5510 - accuracy: 0.7313
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5483 - accuracy: 0.7316
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5475 - accuracy: 0.7327
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5463 - accuracy: 0.7329
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5456 - accuracy: 0.7334
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5452 - accuracy: 0.7331
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5446 - accuracy: 0.7346
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5435 - accura

In [66]:
# Evaluate the model using the testing data

model_loss, model_accuracy = nn_model.evaluate(X_test_scaled, y_test, verbose = 2)

print(f'Loss: {round(model_loss * 100, 2)}%, Accuracy: {round(model_accuracy * 100, 2)}%')

268/268 - 0s - loss: 0.5879 - accuracy: 0.7279 - 440ms/epoch - 2ms/step
Loss: 58.79%, Accuracy: 72.79%


In [67]:
# Save and export results as HDF5 file

nn_model.save("AlphabetSoupCharity.h5")